# Importing packages and function definition 


In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import inv
import sklearn
from sklearn import datasets
from sklearn.datasets import load_iris
from sklearn.datasets import load_digits
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.covariance import OAS
from scipy.stats import bartlett

def fold_k(X,y,k,G, func):
    fold = np.random.choice(np.arange(5), len(y))
    Xtrain, ytrain = X[fold != k, :], y[fold != k]
    Xtest, ytest = X[fold == k, :], y[fold == k]
    scaler = MinMaxScaler()
    scaler.fit(Xtrain)
    Xtrain = scaler.transform(Xtrain)
    Xtest = scaler.transform(Xtest)
    return func(Xtrain, ytrain, Xtest, ytest, G)

def dist(u,v,T):
  distance = np.dot((u - v).flatten(), np.matmul(u-v, np.linalg.inv(T)).flatten())
  return distance

def tradi_lda(X,y, Xtest, ytest, G):
  # le2 = LabelEncoder()
  # y = le2.fit_transform(y)
  W = np.array([sum(y==g)*np.cov(X[y==g,:].T) for g in range(G)])
  W = np.sum(W, axis=0)
  xbar = np.mean(X, axis = 0) 
  xi_bar = np.array([np.mean(X[y==g,:], axis=0) for g in range(G)])
  B = np.array([sum(y==g)*np.outer(xi_bar[g,:]-xbar,xi_bar[g,:]-xbar.T) for g in range(G)])
  B = np.sum(B, axis = 0)
  eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(W).dot(B))
  eig_vecs = np.real(eig_vecs)

  # prediction
  n_take = sum(np.abs(np.real(eig_vals)) >0.000001)# number of eigvalues chosen  
  f = lambda x, k: sum(np.array([(eig_vecs[:,r].dot(x - xi_bar[k,:]))**2 for r in range(n_take)]))
  score = lambda i: np.array([f(Xtest[i,:],k) for k in range(G)])
  ypred = np.array([np.argmin(score(i)) for i in range(len(ytest))])  
  # ypred = le2.inverse_transform(ypred)
  return sum(ypred !=ytest)/len(ytest)#confusion_matrix(ytest, ypred)

In [2]:
def shrink_lda(X,y, Xtest, ytest, G):
  le2 = LabelEncoder()
  y = le2.fit_transform(y)
  W = np.array([sum(y==g)*OAS().fit(X[y==g,:]).covariance_ for g in range(G)])
  W = np.sum(W, axis=0)
  xbar = np.mean(X, axis = 0) 
  xi_bar = np.array([np.mean(X[y==g,:], axis=0) for g in range(G)])
  B = np.array([sum(y==g)*np.outer(xi_bar[g,:]-xbar,xi_bar[g,:]-xbar.T) for g in range(G)])
  B = np.sum(B, axis = 0)
  eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(W).dot(B))
  eig_vecs = np.real(eig_vecs)

  # prediction
  n_take = sum(np.abs(np.real(eig_vals)) >0.000001)
  f = lambda x, k: sum(np.array([(eig_vecs[:,r].dot(x - xi_bar[k,:]))**2 for r in range(n_take)]))
  score = lambda i: np.array([f(Xtest[i,:],k) for k in range(G)])
  ypred = np.array([np.argmin(score(i)) for i in range(len(ytest))])  
  ypred = le2.inverse_transform(ypred)
  return sum(ypred !=ytest)/len(ytest)#confusion_matrix(ytest, ypred)

# VarLDA

In [3]:
def varLDA(X,y,Xtest,ytest,G):
  W = np.array([sum(y==g)*np.cov(X[y==g,:].T) for g in range(G)])
  W = np.sum(W, axis=0)
  xbar = np.mean(X, axis = 0) 
  xi_bar = np.array([np.mean(X[y==g,:], axis=0) for g in range(G)])
  B = np.array([sum(y==g)*np.outer(xi_bar[g,:]-xbar,xi_bar[g,:]-xbar.T) for g in range(G)])
  B = np.sum(B, axis = 0)
  eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(W).dot(B))
  eig_vecs = np.real(eig_vecs)

  n_take = sum(np.abs(np.real(eig_vals)) >0.000001)
  # compute class variances in transformed space
  space_i_var = lambda i:  np.array([eig_vecs[:,i].dot(
      np.matmul(np.cov(X[y==g,:].T),eig_vecs[:,i].T)) for g in range(G)])
  vars = np.array([space_i_var(i) for i in range(n_take)])
  # print('vars:\n', vars)

  # prediction
  f = lambda x, k: sum(np.array([(eig_vecs[:,r].dot(x - xi_bar[k,:]))**2/vars[r,k] for r in range(n_take)]))
  score = lambda i: np.array([f(Xtest[i,:],k) for k in range(G)])
  ypred = np.array([np.argmin(score(i)) for i in range(len(ytest))])  
  return sum(ypred !=ytest)/len(ytest)#confusion_matrix(ytest, ypred)  

def qda(Xtrain, ytrain, Xtest, ytest, G):
  clf = QuadraticDiscriminantAnalysis()
  clf.fit(Xtrain, ytrain)
  return sum(clf.predict(Xtest)!=ytest)/len(ytest)

In [4]:
def varShrinkLDA(X,y,Xtest,ytest,G):
  le2 = LabelEncoder()
  y = le2.fit_transform(y)  
  W = np.array([sum(y==g)*OAS().fit(X[y==g,:]).covariance_ for g in range(G)])
  W = np.sum(W, axis=0)
  xbar = np.mean(X, axis = 0) 
  xi_bar = np.array([np.mean(X[y==g,:], axis=0) for g in range(G)])
  B = np.array([sum(y==g)*np.outer(xi_bar[g,:]-xbar,xi_bar[g,:]-xbar.T) for g in range(G)])
  B = np.sum(B, axis = 0)
  eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(W).dot(B))
  eig_vecs = np.real(eig_vecs)

  n_take = sum(np.abs(np.real(eig_vals)) >0.000001)
  # compute class variances in transformed space
  space_i_var = lambda i:  np.array([eig_vecs[:,i].dot(
      np.matmul(np.cov(X[y==g,:].T),eig_vecs[:,i].T)) for g in range(G)])
  vars = np.array([space_i_var(i) for i in range(n_take)])
  # print('vars:\n', vars)

  # # prediction
  f = lambda x, k: sum(np.array([(eig_vecs[:,r].dot(x - xi_bar[k,:]))**2/vars[r,k] for r in range(n_take)]))
  score = lambda i: np.array([f(Xtest[i,:],k) for k in range(G)])
  ypred = np.array([np.argmin(score(i)) for i in range(len(ytest))])  
  ypred = le2.inverse_transform(ypred)
  return sum(ypred !=ytest)/len(ytest)#confusion_matrix(ytest, ypred)  


# Lp LDA

In [5]:
import random
#Based on the paper "Generalization of linear discriminant analysis using Lp-norm" by J.H.Oh and N.Kwak
def sign_function(a):
    if (a>0):
       return 1
    elif (a==0):
       return 0
    else:
       return -1

def singularity_condition(X,y,w_t,m,m_c):
    #The output is 1 (i.e True) if: w(t).T*(m_c-m) = 0 
    #or w(t).T*(x_i-m_c_i) = 0 for some x_i
    for i in range(len(X)):
        if ((np.dot(w_t,(X[i]-m_c[y[i]]))==0) or (np.dot(w_t, (m_c[y[i]] - m))==0)): 
           return 1
    return 0

def get_gradient(p,X,y,w_t,m,m_c,G):
   
    A = np.array([sum(y==g)*sign_function(np.dot(w_t,(m_c[g]-m)))*abs(np.dot(w_t,(m_c[g]-m)))**(p-1)*(m_c[g]-m) for g in range(G)])
    A = p*np.sum(A, axis=0)
    
    B = np.array([(sign_function(np.dot(w_t,(X[i]-m_c[y[i]])))*np.dot(w_t,(X[i]-m_c[y[i]])))**p for i in range(len(X))])
    B = np.sum(B,axis=0)

    C = np.array([sum(y==g)*(sign_function(np.dot(w_t,(m_c[g]-m)))*np.dot(w_t,(m_c[g]-m)))**p for g in range(G)])
    C = np.sum(C,axis=0)

    D = np.array([sign_function(np.dot(w_t,(X[i]-m_c[y[i]])))*abs(np.dot(w_t,(X[i]-m_c[y[i]])))**(p-1)*(X[i]-m_c[y[i]]) for i in range(len(X))])
    D = p*np.sum(D,axis=0)

    E = np.array([(sign_function(np.dot(w_t,(X[i]-m_c[y[i]])))*np.dot(w_t,(X[i]-m_c[y[i]])))**p for i in range(len(X))])
    E = (np.sum(E,axis=0))**2

    return (A*B - C*D)/E
       

def lda_lp_one_vector(X, y, G, p, learning_rate, epsilon):
    le2 = LabelEncoder()
    y = le2.fit_transform(y)
    alpha = learning_rate

    #Initialization
    t=0
    N, D = np.shape(X) 
    w_t = [random.uniform(0.1,0.5) for _ in range(D)] #set an abitrary vector w_t such that ||w_t||_2 = 1
    w_t = w_t/np.linalg.norm(w_t)        
    
    m = np.mean(X, axis = 0)
    m_c = np.array([np.mean(X[y==g,:], axis=0) for g in range(G)])

    while True:

          if (p<=1):
             #Singularity check
             if (singularity_condition(X,y,w_t,m,m_c)==1): 
                 #w_t <- w_t + delta/||w_t + delta|| for delta is a small random vector
                 n_del = D #(= len(w_t))
                 delta = [random.uniform(1e-6,1e-5) for _ in range(n_del)]
                 w_t = (w_t + delta)/np.linalg.norm(w_t + delta)

          dw = get_gradient(p,X,y,w_t,m,m_c,G)

          w_t_0 = w_t       
          w_t = w_t + alpha*dw #or "-" ??
          w_t = w_t/np.linalg.norm(w_t)
          if (np.linalg.norm(w_t - w_t_0)<epsilon): break

    return w_t      

def lda_lp_multiple_vectors(X, y, G, p, learning_rate, epsilon, number_of_vect):
    w_j = 0.0
    X_j = X
    w = []
    for j in range(number_of_vect):
        for i in range(len(X)):
            X_j[i] = X_j[i] - w_j*np.dot(w_j,X_j[i])
        w_j =  lda_lp_one_vector(X_j, y, G, p, learning_rate, epsilon)
        w.append(w_j)
    return np.array(w)   

In [6]:
def lda_lp_original(X, y, Xtest, ytest, G, p=1.5,number_of_vect = 1, learning_rate = 0.0001, epsilon=1e-5): #1e-6
  n_take = number_of_vect #(i.e Number of projection vectors)
  w = lda_lp_multiple_vectors(X,y,G,p,learning_rate,epsilon,number_of_vect)
  m = np.mean(X, axis = 0)
  m_c = np.array([np.mean(X[y==g,:], axis=0) for g in range(G)])

  n_take = number_of_vect
  f = lambda x, k: sum(np.array([(w[r,:].dot(x - m_c[k,:]))**2 for r in range(n_take)]))
  score = lambda i: np.array([f(Xtest[i,:],k) for k in range(G)])
  ypred = np.array([np.argmin(score(i)) for i in range(len(ytest))])  
  #ypred = le2.inverse_transform(ypred)
  return sum(ypred !=ytest)/len(ytest)#confusion_matrix(ytest, ypred)

In [12]:
def lda_lp(X, y, Xtest, ytest, G, p=1.5,number_of_vect = 1, learning_rate = 0.0001, epsilon=1e-5): #NOTE: this is UC-lpLDA
  n_take = number_of_vect #(i.e Number of projection vectors)
  w = lda_lp_multiple_vectors(X,y,G,p,learning_rate,epsilon,number_of_vect)
  m = np.mean(X, axis = 0)
  m_c = np.array([np.mean(X[y==g,:], axis=0) for g in range(G)])

  # compute class variances in transformed space
  space_i_var = lambda i:  np.array([w[i,:].dot(
      np.matmul(np.cov(X[y==g,:].T),w[i,:].T)) for g in range(G)])
  vars = np.array([space_i_var(i) for i in range(n_take)])
  sd = np.sqrt(vars)
  # print('vars:\n', vars)

  n_take = number_of_vect
  f = lambda x, k: sum(np.array([np.abs(w[r,:].dot(x - m_c[k,:]))/sd[r,k] for r in range(n_take)]))
  score = lambda i: np.array([f(Xtest[i,:],k) for k in range(G)])
  ypred = np.array([np.argmin(score(i)) for i in range(len(ytest))])  
  #ypred = le2.inverse_transform(ypred)
  return sum(ypred !=ytest)/len(ytest)#confusion_matrix(ytest, ypred)

# heart 

https://archive.ics.uci.edu/ml/datasets/SPECTF+Heart


In [13]:
data = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECTF.train', header = None,sep=',')
data.head()
test = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECTF.test',
                     header=None, sep = ',')
data = pd.concat([data, test])
data = data.to_numpy()
X,y = data[:,1:], data[:,0]
G = len(np.unique(y)) 
print(np.shape(X))
for g in range(G):
  print(sum(y==g))

(267, 44)
55
212


In [17]:
np.random.seed(1)
tradi_res = [fold_k(X,y,k,G, tradi_lda) for k in np.arange(5)]
varlda_res = [fold_k(X,y,k,G, varLDA) for k in np.arange(5)]
b = np.mean(np.asarray(tradi_res), axis=0)
c = np.mean(np.asarray(varlda_res), axis=0)
b,c

(0.34435137042968716, 0.2560807069442864)

In [11]:
K = 5
varSrhinklda_res = [fold_k(X,y,k,G, varShrinkLDA) for k in np.arange(K)]
shrinklda_res = [fold_k(X,y,k,G, shrink_lda) for k in np.arange(K)]
print(np.mean(np.asarray(varSrhinklda_res), axis=0), np.mean(np.asarray(shrinklda_res), axis=0))
np.mean(np.asarray(varSrhinklda_res), axis=0)- np.mean(np.asarray(shrinklda_res), axis=0)

0.2616084671940908 0.3587995741073132


-0.09719110691322241

In [21]:
qda_res = [fold_k(X,y,k,G, qda) for k in np.arange(5)]
d = np.mean(np.asarray(qda_res), axis=0)
d

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.2081674351258692

In [18]:
lplda_res = [fold_k(X,y,k,G, lda_lp) for k in np.arange(5)]
# print(lplda_res)
d = np.mean(np.asarray(lplda_res), axis=0)
d

0.2724116403220881

In [83]:
lplda_res = [fold_k(X,y,k,G, lda_lp_original) for k in np.arange(5)]
# print(lplda_res)
d = np.mean(np.asarray(lplda_res), axis=0)
d

0.3373304334400945

# Car 

In [19]:
#source: https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn

from sklearn.pipeline import Pipeline

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [20]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data',
                  sep = ",", header = None)
for i in range(6):
  print(np.unique(data[[i]]))
data = MultiColumnLabelEncoder(columns = [0,1,2,3,4,5,6]).fit_transform(data)
print(data.head())
data = data.to_numpy()
X,y = data[:, [x for x in range(data.shape[1]) if x != 6]].astype(np.float32),data[:,-1]

G = len(np.unique(y))
le2 = LabelEncoder()
y = le2.fit_transform(y)
for g in range(G):
  print(sum(y==g))

['high' 'low' 'med' 'vhigh']
['high' 'low' 'med' 'vhigh']
['2' '3' '4' '5more']
['2' '4' 'more']
['big' 'med' 'small']
['high' 'low' 'med']
   0  1  2  3  4  5  6
0  3  3  0  0  2  1  2
1  3  3  0  0  2  2  2
2  3  3  0  0  2  0  2
3  3  3  0  0  1  1  2
4  3  3  0  0  1  2  2
384
69
1210
65


In [21]:
np.random.seed(1)
tradi_res = [fold_k(X,y,k,G, tradi_lda) for k in np.arange(5)]
varlda_res = [fold_k(X,y,k,G, varLDA) for k in np.arange(5)]
b = np.mean(np.asarray(tradi_res), axis=0)
c = np.mean(np.asarray(varlda_res), axis=0)
b,c

(0.5275899914866666, 0.3964788690978383)

In [ ]:
varSrhinklda_res = [fold_k(X,y,k,G, varShrinkLDA) for k in np.arange(K)]
shrinklda_res = [fold_k(X,y,k,G, shrink_lda) for k in np.arange(K)]
np.mean(np.asarray(varSrhinklda_res), axis=0), np.mean(np.asarray(shrinklda_res), axis=0)

(0.38627138454410814, 0.5014950379507386)

In [25]:
qda_res = [fold_k(X,y,k,G, qda) for k in np.arange(5)]
d = np.mean(np.asarray(qda_res), axis=0)
d

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:833: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:833: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:836: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:833: RuntimeWarning: divide by zero encountered in power
  X2 = np.

0.9625860093684422

In [22]:
lplda_res = [fold_k(X,y,k,G, lda_lp) for k in np.arange(5)]
# print(lplda_res)
d = np.mean(np.asarray(lplda_res), axis=0)
d

0.48715741265679996

In [86]:
lplda_res = [fold_k(X,y,k,G, lda_lp_original) for k in np.arange(5)]
# print(lplda_res)
d = np.mean(np.asarray(lplda_res), axis=0)
d

0.6247749958694064

# balance 

In [23]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/balance-scale/balance-scale.data',
                  sep = ",", header = None)
data.head()

data = data.to_numpy()
X,y = data[:, [x for x in range(data.shape[1]) if x != 0]].astype(np.float32),data[:,0]
G = len(np.unique(y))
le2 = LabelEncoder()
y = le2.fit_transform(y)
for g in range(G):
  print(sum(y==g))
print(X.shape)

49
288
288
(625, 4)


In [24]:
np.random.seed(1)
tradi_res = [fold_k(X,y,k,G, tradi_lda) for k in np.arange(5)]
varlda_res = [fold_k(X,y,k,G, varLDA) for k in np.arange(5)]
a,b = np.mean(np.asarray(tradi_res), axis=0), np.mean(np.asarray(varlda_res), axis=0)
a,b

(0.2708869839937164, 0.09230761381826066)

In [ ]:
K = 5
varSrhinklda_res = [fold_k(X,y,k,G, varShrinkLDA) for k in np.arange(K)]
shrinklda_res = [fold_k(X,y,k,G, shrink_lda) for k in np.arange(K)]
np.mean(np.asarray(varSrhinklda_res), axis=0), np.mean(np.asarray(shrinklda_res), axis=0)

(0.0839318787427194, 0.25779190323484136)

In [27]:
qda_res = [fold_k(X,y,k,G, qda) for k in np.arange(5)]
d = np.mean(np.asarray(qda_res), axis=0)
d

0.0839318787427194

In [ ]:
def fold_k_qda(X,y,k,G):
    np.random.seed(1)
    fold = np.random.choice(np.arange(5), len(y))
    Xtrain, ytrain = X[fold != k, :], y[fold != k]
    Xtest, ytest = X[fold == k, :], y[fold == k]
    scaler = MinMaxScaler()
    scaler.fit(Xtrain)
    Xtrain = scaler.transform(Xtrain)
    Xtest = scaler.transform(Xtest)
    cls = Kfda(kernel='sigmoid', n_components=2)
    cls.fit(Xtrain, ytrain)
    test_score = cls.score(Xtest, ytest)
    return 1-test_score #func(Xtrain, ytrain, Xtest, ytest, G)

def fold_k_uc_qda(X,y,k,G):
    np.random.seed(1)
    fold = np.random.choice(np.arange(5), len(y))
    Xtrain, ytrain = X[fold != k, :], y[fold != k]
    Xtest, ytest = X[fold == k, :], y[fold == k]
    scaler = MinMaxScaler()
    scaler.fit(Xtrain)
    Xtrain = scaler.transform(Xtrain)
    Xtest = scaler.transform(Xtest)
    cls = Kfda(kernel='sigmoid', n_components=2)
    cls.fit(Xtrain, ytrain)
    ypred = cls.predict_varkda(Xtest)
    print(ypred)
    return sum(ypred!=ytest)/len(ytest) #func(Xtrain, ytrain, Xtest, ytest, G)

In [ ]:
kda_res = [fold_k_qda(X,y,k,G) for k in np.arange(5)]
# varkda_res = [fold_k_uc_qda(X,y,k,G) for k in np.arange(5)]
np.mean(np.asarray(kda_res), axis=0)
# np.mean(np.asarray(varkda_res), axis=0)

0.19553763700147783

In [25]:
lplda_res = [fold_k(X,y,k,G, lda_lp) for k in np.arange(5)]
# print(lplda_res)
d = np.mean(np.asarray(lplda_res), axis=0)
d

0.22949267426953157

In [88]:
lplda_res = [fold_k(X,y,k,G, lda_lp_original) for k in np.arange(5)]
# print(lplda_res)
d = np.mean(np.asarray(lplda_res), axis=0)
d

0.31253305303197737

# Breast tissue 
https://archive.ics.uci.edu/ml/datasets/Breast+Tissue

In [26]:
data = pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00192/BreastTissue.xls',
                     sheet_name=1)
print(data.head())
data = data.to_numpy()
X, y = data[:,2:].astype(np.float32), data[:,1]
G = len(np.unique(y))
le2 = LabelEncoder()
y = le2.fit_transform(y)
print(X.shape)

   Case # Class          I0     PA500       HFS          DA          Area  \
0       1   car  524.794072  0.187448  0.032114  228.800228   6843.598481   
1       2   car  330.000000  0.226893  0.265290  121.154201   3163.239472   
2       3   car  551.879287  0.232478  0.063530  264.804935  11888.391827   
3       4   car  380.000000  0.240855  0.286234  137.640111   5402.171180   
4       5   car  362.831266  0.200713  0.244346  124.912559   3290.462446   

        A/DA     Max IP          DR           P  
0  29.910803  60.204880  220.737212  556.828334  
1  26.109202  69.717361   99.084964  400.225776  
2  44.894903  77.793297  253.785300  656.769449  
3  39.248524  88.758446  105.198568  493.701814  
4  26.342127  69.389389  103.866552  424.796503  
(106, 9)


In [27]:
np.random.seed(1)
tradi_res = [fold_k(X,y,k,G, tradi_lda) for k in np.arange(5)]
varlda_res = [fold_k(X,y,k,G, varLDA) for k in np.arange(5)]
b = np.mean(np.asarray(tradi_res), axis=0)
c = np.mean(np.asarray(varlda_res), axis=0)
b,c

(0.43310343206695345, 0.3821538461538461)

In [ ]:
varSrhinklda_res = [fold_k(X,y,k,G, varShrinkLDA) for k in np.arange(K)]
shrinklda_res = [fold_k(X,y,k,G, shrink_lda) for k in np.arange(K)]
np.mean(np.asarray(varSrhinklda_res), axis=0), np.mean(np.asarray(shrinklda_res), axis=0)

(0.294113810741688, 0.3686713554987212)

In [29]:
qda_res = [fold_k(X,y,k,G, qda) for k in np.arange(5)]
d = np.mean(np.asarray(qda_res), axis=0)
d

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.3878375959079284

In [ ]:
lplda_res = [fold_k(X,y,k,G, lda_lp) for k in np.arange(5)]
# print(lplda_res)
d = np.mean(np.asarray(lplda_res), axis=0)
d

In [98]:
lplda_res = [fold_k(X,y,k,G, lda_lp_original) for k in np.arange(5)]
# print(lplda_res)
d = np.mean(np.asarray(lplda_res), axis=0)
d

0.5403286445012788

# Digits 

In [ ]:
digits = datasets.load_digits()
X,y = digits.data, digits.target.ravel() 
#print(X.shape)
rmid = np.where(sum(X!=0)<10)
X = np.delete(X, rmid,axis = 1)
X.shape, sum(y==0)
G = 10

In [ ]:
np.random.seed(1)
tradi_res = [fold_k(X,y,k,G, tradi_lda) for k in np.arange(5)]
varlda_res = [fold_k(X,y,k,G, varLDA) for k in np.arange(5)]
b = np.mean(np.asarray(tradi_res), axis=0)
c = np.mean(np.asarray(varlda_res), axis=0)
b,c

In [31]:
qda_res = [fold_k(X,y,k,G, qda) for k in np.arange(5)]
d = np.mean(np.asarray(qda_res), axis=0)
d

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.12167676524058242

In [ ]:
varSrhinklda_res = [fold_k(X,y,k,G, varShrinkLDA) for k in np.arange(K)]
shrinklda_res = [fold_k(X,y,k,G, shrink_lda) for k in np.arange(K)]
np.mean(np.asarray(varSrhinklda_res), axis=0), np.mean(np.asarray(shrinklda_res), axis=0)

(0.04928185492370697, 0.04729662971657904)

In [ ]:
lplda_res = [fold_k(X,y,k,G, lda_lp_original) for k in np.arange(5)]
# print(lplda_res)
d = np.mean(np.asarray(lplda_res), axis=0)
d

In [105]:
lplda_res = [fold_k(X,y,k,G, lda_lp) for k in np.arange(5)]
# print(lplda_res)
d = np.mean(np.asarray(lplda_res), axis=0)
d

0.6301090077636385

# Seed 

In [ ]:
data = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt',
                     sep = '\s+', header = None)
print(data.head())
data = pd.DataFrame.to_numpy(data)
# reset the labels to go start from 0  
X,y = data[:,:7], data[:,7]-1 
y = y.astype(np.int32)
G = len(np.unique(y))
print(X.shape)

In [ ]:
np.random.seed(1)
tradi_res = [fold_k(X,y,k,G, tradi_lda) for k in np.arange(5)]
varlda_res = [fold_k(X,y,k,G, varLDA) for k in np.arange(5)]
b = np.mean(np.asarray(tradi_res), axis=0)
c = np.mean(np.asarray(varlda_res), axis=0)
b,c

In [ ]:
varSrhinklda_res = [fold_k(X,y,k,G, varShrinkLDA) for k in np.arange(5)]
shrinklda_res = [fold_k(X,y,k,G, shrink_lda) for k in np.arange(5)]
np.mean(np.asarray(varSrhinklda_res), axis=0), np.mean(np.asarray(shrinklda_res), axis=0)

(0.03851079826812891, 0.09205473914249165)

In [33]:
qda_res = [fold_k(X,y,k,G, qda) for k in np.arange(5)]
d = np.mean(np.asarray(qda_res), axis=0)
d

0.05901955727010048

In [ ]:
lplda_res = [fold_k(X,y,k,G, lda_lp) for k in np.arange(5)]
# print(lplda_res)
d = np.mean(np.asarray(lplda_res), axis=0)
d

In [90]:
lplda_res = [fold_k(X,y,k,G, lda_lp_original) for k in np.arange(5)]
# print(lplda_res)
d = np.mean(np.asarray(lplda_res), axis=0)
d

0.20553795278576384

# Iris

In [ ]:
iris = datasets.load_iris()
X,y = iris.data, iris.target.ravel() 
G = 3

In [ ]:
np.random.seed(2)
tradi_res = [fold_k(X,y,k,G, tradi_lda) for k in np.arange(5)]
varlda_res = [fold_k(X,y,k,G, varLDA) for k in np.arange(5)]
b = np.mean(np.asarray(tradi_res), axis=0)
c = np.mean(np.asarray(varlda_res), axis=0)
b,c

In [101]:
qda_res = [fold_k(X,y,k,G, qda) for k in np.arange(5)]
d = np.mean(np.asarray(qda_res), axis=0)
d

0.014285714285714285

In [81]:
varSrhinklda_res = [fold_k(X,y,k,G, varShrinkLDA) for k in np.arange(5)]
shrinklda_res = [fold_k(X,y,k,G, shrink_lda) for k in np.arange(5)]
np.mean(np.asarray(varSrhinklda_res), axis=0), np.mean(np.asarray(shrinklda_res), axis=0)

(0.0288637194008038, 0.03652564102564103)

In [ ]:
lplda_res = [fold_k(X,y,k,G, lda_lp) for k in np.arange(5)]
# print(lplda_res)
d = np.mean(np.asarray(lplda_res), axis=0)
d

In [98]:
lplda_res = [fold_k(X,y,k,G, lda_lp_original) for k in np.arange(5)]
# print(lplda_res)
d = np.mean(np.asarray(lplda_res), axis=0)
d

0.02641931216931217



# Wine better 3%, 0%

In [ ]:
wine = datasets.load_wine()
X,y = wine.data, wine.target.ravel() 
G = 3
X.shape

In [ ]:
np.random.seed(2)
tradi_res = [fold_k(X,y,k,G, tradi_lda) for k in np.arange(5)]
varlda_res = [fold_k(X,y,k,G, varLDA) for k in np.arange(5)]
b = np.mean(np.asarray(tradi_res), axis=0)
c = np.mean(np.asarray(varlda_res), axis=0)
b,c

In [106]:
qda_res = [fold_k(X,y,k,G, qda) for k in np.arange(5)]
d = np.mean(np.asarray(qda_res), axis=0)
d

0.017152961980548186

In [90]:
K = 5
varSrhinklda_res = [fold_k(X,y,k,G, varShrinkLDA) for k in np.arange(K)]
shrinklda_res = [fold_k(X,y,k,G, shrink_lda) for k in np.arange(K)]
np.mean(np.asarray(varSrhinklda_res), axis=0), np.mean(np.asarray(shrinklda_res), axis=0)

(0.23140225787284613, 0.3018885448916409)

In [ ]:
lplda_res = [fold_k(X,y,k,G, lda_lp) for k in np.arange(5)]
# print(lplda_res)
d = np.mean(np.asarray(lplda_res), axis=0)
d

In [92]:
lplda_res = [fold_k(X,y,k,G, lda_lp_original) for k in np.arange(5)]
# print(lplda_res)
d = np.mean(np.asarray(lplda_res), axis=0)
d

0.06048245906004527